In [1]:
%matplotlib inline
path = "../data/dl1/dogscats/"
# path = "../data/dl1/dogscats/sample/"
#TYPE = 'train'
TYPE = 'valid'
SEED = 1101

In [2]:
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import scipy.spatial.distance
import utils; reload(utils)
from utils import plots
from time import time

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [3]:
# import vgg class
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [4]:
# define submodels to extract representations at a given layer
from keras.models import Model
idx = [0,5,10,17,24,31,33,35,37]

# lambda_1
# maxpooling2d_1
# maxpooling2d_2
# maxpooling2d_3
# maxpooling2d_4
# maxpooling2d_5
# dense_1
# dense_2
# dense_3

def get_models(idx,base):
    layers  = []
    for i in range(len(idx)):
        print(base.layers[idx[i]].name)
        layers.append( base.layers[idx[i]] )

    models = []
    outshapes = []
    for layer in layers:
        models.append( Model(input=base.input,
                             output=base.get_layer(layer.name).output) )
        outshapes.append(list( layer.output_shape ))
        
        
    print('Done.')
    return layers,models,outshapes

In [5]:
# instantiate base model and sub-models
vgg = Vgg16()
base = vgg.model
layers,models,outshapes = get_models(idx,base)

lambda_1
maxpooling2d_1
maxpooling2d_2
maxpooling2d_3
maxpooling2d_4
maxpooling2d_5
dense_1
dense_2
dense_3
Done.


### Extract representations without finetuning

In [6]:
# I extract the representations of 2000 images in the training or validation set (TYPE)
datapath = path + TYPE + '/'
repath = '../data/dl1/objnc/rep/dogscats/nofinetune/'
batch_size = 50
nbatches   = 40
print('N.of batches ' + str(nbatches))


N.of batches 40


In [7]:
# get labels for future use
batches = vgg.get_batches(datapath, batch_size=batch_size, shuffle=True, seed=SEED)
L = []
for n in range(nbatches):
    _,labels = batches.next()      
    L.append(labels)
L = np.asarray(L, dtype=float) 
L.shape = (nbatches*batch_size,2)
np.save(repath +  'labels' + '_' + TYPE, L)

Found 2000 images belonging to 2 classes.


In [8]:
# For each batch extract representations. Notice that in case TYPE is 'train' you need to
# set the SEED in get_batches to enforce reproducibility. 
# This is not needed when TYPE is 'valid' since there we use the whole validation data but we use it
# anyway. In this way we can separately capture the images and correct labels for future use, using
# the same batch_size

D = []
D_sq = []
for layer,model in zip(layers,models):
    print('Processing model : ' + layer.name)
    R = []
    batches = vgg.get_batches(datapath, batch_size=batch_size, shuffle=True, seed=SEED)
    ti = time()
    for n in range(nbatches):       
        imgs,labels = batches.next()       
        R.append(model.predict(imgs))
    R = np.asarray(R, dtype=float) 
    outshape = list(layer.output_shape)
    outshape[0] = nbatches*batch_size
    R.shape = tuple(outshape)
    print(R.shape)
    np.save(repath +  layer.name + '_' + TYPE, R)
    
    # compute distances
    R.shape = R.shape[0], -1
    d = scipy.spatial.distance.pdist(R, 'euclidean')
    d_sq = scipy.spatial.distance.squareform(d, force='no', checks=True)
    D.append(d)
    D_sq.append(d_sq)
    te = time() - ti
    print(te) 
    
np.save(repath + 'D'    + '_' + TYPE, D)
np.save(repath + 'D_sq' + '_' + TYPE, D_sq)
print('Done.')

Processing model : lambda_1
Found 2000 images belonging to 2 classes.
(2000, 3, 224, 224)
408.567681074
Processing model : maxpooling2d_1
Found 2000 images belonging to 2 classes.
(2000, 64, 112, 112)
2129.75369287
Processing model : maxpooling2d_2
Found 2000 images belonging to 2 classes.
(2000, 128, 56, 56)
1053.20662999
Processing model : maxpooling2d_3
Found 2000 images belonging to 2 classes.
(2000, 256, 28, 28)
563.425999165
Processing model : maxpooling2d_4
Found 2000 images belonging to 2 classes.
(2000, 512, 14, 14)
317.03707099
Processing model : maxpooling2d_5
Found 2000 images belonging to 2 classes.
(2000, 512, 7, 7)
134.26051712
Processing model : dense_1
Found 2000 images belonging to 2 classes.
(2000, 4096)
83.6617281437
Processing model : dense_2
Found 2000 images belonging to 2 classes.
(2000, 4096)
84.3319208622
Processing model : dense_3
Found 2000 images belonging to 2 classes.
(2000, 1000)
77.0289649963
Done.


### Finetune

In [9]:
# define batches, now this is not constrained to be 50 anymore. We use these batches for training only
batch_size = 64
batches = vgg.get_batches(path+'train', batch_size=batch_size, seed=SEED)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size, seed=SEED)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
vgg.finetune(batches)

In [11]:
vgg.fit(batches, val_batches, nb_epoch=5)

Epoch 1/5
23000/23000 [==============================] - 650s - loss: 0.1188 - acc: 0.9697 - val_loss: 0.0795 - val_acc: 0.9785
Epoch 2/5
23000/23000 [==============================] - 651s - loss: 0.0966 - acc: 0.9772 - val_loss: 0.0674 - val_acc: 0.9860
Epoch 3/5
23000/23000 [==============================] - 652s - loss: 0.1048 - acc: 0.9774 - val_loss: 0.0875 - val_acc: 0.9810
Epoch 4/5
23000/23000 [==============================] - 652s - loss: 0.1025 - acc: 0.9789 - val_loss: 0.0807 - val_acc: 0.9850
Epoch 5/5
23000/23000 [==============================] - 652s - loss: 0.1021 - acc: 0.9801 - val_loss: 0.0786 - val_acc: 0.9850


##### Observation 
I notice that it's doing strange things but this is not important since we need just to compute anew the representations of the last hidden layer - the new one inserted to substitute the last hidden layer in the original VGG

In [12]:
repath   = '../data/dl1/objnc/rep/dogscats/finetune/'
vgg.model.save_weights(repath + 'w_ft.h5')

### Extract representations last hidden layer finetuned

In [13]:
base = vgg.model
layers,models,outshapes = get_models(idx,base)

lambda_1
maxpooling2d_1
maxpooling2d_2
maxpooling2d_3
maxpooling2d_4
maxpooling2d_5
dense_1
dense_2
dense_4
Done.


In [14]:
datapath = path + TYPE + '/'
batch_size = 50
nbatches   = 40
batches = vgg.get_batches(datapath, batch_size=batch_size, shuffle=True, seed = SEED)
print('N.of batches ' + str(nbatches))

Found 2000 images belonging to 2 classes.
N.of batches 40


In [15]:
repath = '../data/dl1/objnc/rep/dogscats/finetune/'

D = []
D_sq = []
layer = layers[-1]
model = models[-1]
print('Processing model : ' + layer.name)

R = []
ti = time()
for n in range(nbatches):
    imgs,_ = batches.next()       
    R.append(model.predict(imgs))
R = np.asarray(R, dtype=float) 
outshape = list(layer.output_shape)
outshape[0] = nbatches*batch_size
R.shape = tuple(outshape)
print(R.shape)
np.save(repath +  layer.name + '_' + TYPE, R)

# compute distances
R.shape = R.shape[0], -1
d = scipy.spatial.distance.pdist(R, 'euclidean')
d_sq = scipy.spatial.distance.squareform(d, force='no', checks=True)
D.append(d)
D_sq.append(d_sq)
te = time() - ti
print(te) 
    
np.save(repath + 'D'    + '_' + TYPE, D)
np.save(repath + 'D_sq' + '_' + TYPE, D_sq)
print('Done.')

Processing model : dense_4
(2000, 2)
74.9427008629
Done.
